## Segmenting and Clustering Neighborhoods in Toronto
### Peer-Graded Assignment 
### Part 1 

## Introduction

For this assignment, we were required to explore and cluster the neighborhoods in Toronto.For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto, The data given were in tabular format consisting of three columns -"Postal Code", "Borough", "Neighborhood".
The objective was to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format.
Once the data is in a structured format,we can easily explore and cluster the neighborhoods in the city of Toronto.

WIKIPEDIA LINK - "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

## Libraries Required

### BeautifulSoup
Beautiful Soup is a Python library for pulling data out of HTML and XML files. It works with your favorite parser to provide idiomatic ways of navigating, searching, and modifying the parse tree. It commonly saves programmers hours or days of work.

### Requests
Requests is a Python HTTP library, released under the Apache License 2.0. The goal of the project is to make HTTP requests simpler and more human-friendly. The current version is 2.23.0.Requests is one of the most popular python libraries that is not included with python, it has been proposed that requests be distributed with python by default.

### Pandas
pandas is a software library written for the Python programming language for data manipulation and analysis. In particular, it offers data structures and operations for manipulating numerical tables and time series. It is free software released under the three-clause BSD license.

### Numpy
Numpy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
print("Required library installed")

Required library installed


## Step 1
## Scrap data from wikipedia 

## Step 1.1
#### 1. Getting data from wikipedia
#### 2.Applying BeautifulSoup to parse html data
#### 3.Storing parsed data into Pandas DataFrame (df)

In [3]:
#get html from wiki page and create soup object
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')

#using soup object, iterate the .wikitable to get the data from the HTML page and store it into a list
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert list into Pandas DataFrame
canada_df = pd.DataFrame(data = data,columns = columns)
canada_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Step 1.2: Data Cleanup
### According to a given conditions
1. Remove Boroughs that are 'Not assigned'
2. More than one neighborhood can exist in one postal code area, combined these into one row with the neighborhoods separated with a comma
3. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [4]:
#Remove Boroughs that are 'Not assigned'
canada_df = canada_df[canada_df['Borough'] != 'Not assigned']
canada_df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [32]:
canada_df = canada_df.groupby(["Postal Code", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
canada_df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
canada_df['Neighborhood'].replace("Not assigned", canada_df["Borough"],inplace=True)
canada_df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Step 1.3- Output
#### Applying ".shape" method

In [6]:
canada_df.shape

(103, 3)

## Step 2 - Adding Geospatial Data

Now that we have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name
Now the next step include utilization of the Foursquare location data,and for that we need to get the latitude and the longitude coordinates of each neighborhood.

We are using a link which is already given in the assignment itself, to a csv file that has the geographical coordinates of each postal code to get the latitude and the longitude coordinates of each neighborhood.
CSV LINK - "http://cocl.us/Geospatial_data ".

In [20]:
#Gatthering data lat/long data from csv
df_geo_coor = pd.read_csv("https://cocl.us/Geospatial_data")
df_geo_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [33]:
df = pd.merge(df_geo_coor, canada_df, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)
df.head()


,Postal Code,Latitude,Longitude,Borough,Neighborhood
0,M8W,43.602414,-79.543484,Etobicoke,"Alderwood, Long Branch"
1,M8V,43.605647,-79.501321,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores"
2,M8Z,43.628841,-79.520999,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
3,M5V,43.628947,-79.394420,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."
4,M8Y,43.636258,-79.498509,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Changing the order of heading

In [35]:
cols = df.columns.tolist()
new_column_order = ['Postal Code',
 'Borough',
 'Neighborhood',
 'Latitude',
 'Longitude']
new_column_order
df = df[new_column_order]
df.head(11)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
1,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321
2,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999
3,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420
4,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
5,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191
6,M7R,Mississauga,Canada Post Gateway Processing Centre,43.636966,-79.615819
7,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
8,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201
9,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
